# TQT: Quantum Photonic Technologies, Example Python Script
This Jupyter notebook demonstrates a minimal script for controlling and measuring a quantum optical experiment.
We will interface with three pieces of experimental hardware: the time-taggers, the laser, and the optical power meter.

In [2]:
from time import sleep

from experiment import QuantumOpticalExperiment
from tqt.utils.io import IO

We establish connections to the devices by creating an instance of the `QuantumOpticalExperiment` class.

In [3]:
experiment = QuantumOpticalExperiment()

FPGA Version: 23005
Resolution: 1.5625e-10
NO Inputs: 16


First, let's see how to control the laser.

There are three control options: turn on, turn off, and change the optical power.

In [4]:
# turn on laser
experiment.laser.on()

# change laser power to 10 mW
experiment.laser.set_power(10)

# wait for 1 second
sleep(1)

# turn off
experiment.laser.off()

b'CMD> \r\n'

Now, let's see how to measure optical power with a Thorlabs power meter. There is only one function of interest, which is to measure the current optical power in mW. If there is no power meter connected, then a value of `NaN` will be returned.

In [5]:
power = experiment.powermeter.get_power()
print(power)

9.69812888e-08


Finally, let's explore how to measure single photon counts. The time tagger how two modes: *logic* mode and *tag* mode. If you are interested only in count rates (singles, coincidences), then you will need to use the *logic* mode. If you require the precise arrival times of photons and on which channels, use the *tag* mode. Let's start with the *logic* mode.

In [6]:
experiment.timetagger.switch_logic()

# collect counts for 1 second
experiment.timetagger.read(time=1)

# extract the total counts and count rate for Channel 1 (i.e., singles on Channel 1)
dt1, counts1, rate1 = experiment.timetagger.get_count_data(channels=[1])

# similarly, for Channel 2
dt2, counts2, rate2 = experiment.timetagger.get_count_data(channels=[2])

# for coincidence counts, we list of channels of interest - e.g., coincidence between Channels 1 & 2
dt12, counts12, rate12 = experiment.timetagger.get_count_data(channels=[1, 2])

print(f"Channel 1: {rate1}, Channel 2: {rate2}, Channel 1 & 2 Coincidences: {rate12}")

Channel 1: 3119.3296273552296, Channel 2: 2153.0888564357438, Channel 1 & 2 Coincidences: 0.0


Now moving to the *tag* mode, we will save a time-tag file for later analysis.

In [9]:
# save time tags to file for a measurement time of 1.0 seconds to a file in "data/time-tags-example/time-tags.txt"
io = IO.directory(folder="time-tags-example")
experiment.timetagger.save_tags(io=io, filename="time-tags", time=1.0, convert=True)
tags = io.load_timetags(filename="time-tags.txt")
print(tags)

100%|██████████| 100/100 [00:01<00:00, 64.13it/s]


31/08/2022, 16:18:20 | Time tag files saved to c:\Users\mjgra\Desktop\Optics_QIC861\python\tqt-optical-control\data\time-tags-example\time-tags.tt
31/08/2022, 16:18:21 | Loaded from c:\Users\mjgra\Desktop\Optics_QIC861\python\tqt-optical-control\data\time-tags-example\time-tags.txt successfully.
[[2.00000000e+00 0.00000000e+00]
 [1.00000000e+00 8.16560000e+04]
 [1.00000000e+00 4.71004000e+05]
 ...
 [2.00000000e+00 9.89123098e+09]
 [2.00000000e+00 9.89145076e+09]
 [1.00000000e+00 9.89296556e+09]]


Once you are finished measuring from the hardware devices, it is good practice to close connections.

In [10]:
# closes connections to all devices
experiment.close()